In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.linear_model import LinearRegression

##### Будем использовать модель линейной регрессии для предсказания стоимости недвижимости. В качестве регрессоров возьмем такие признаки как: 
##### square - площадь, кв.м.
##### number_rooms - количество жилых комнат
##### distance_subway -  расстояние до станции метро, м.
##### year - количество лет с постройки объекта недвижимости, в годах
##### floor - этаж
##### cost_Gazprom_share - стоимость акции Газпрома на момент продажи объекта недвижимости, руб.
##### pet - наличие домашнего питомца у покупателя, dummy - переменная
##### rainfall - толщина осадков за месяц в момент продажи объекта недвижиости, мм.
##### целевая зависимость описывается следующим уравнением: 
##### price_house = 70000 * square + 50000 * number_rooms - 1500 * distance_subway - 3000 * year + 1000 * floor + 60000

In [82]:
n_samples = 1000

square = np.random.choice(45, n_samples) + 20
number_rooms = np.random.choice(2, n_samples) + 3
distance_subway = np.random.choice(np.arange(1,3), n_samples) 
year = np.random.choice(40, n_samples) 
floor = np.random.choice(25, n_samples) 
cost_Gazprom_share = np.random.choice(500, n_samples) + 100
pet = np.random.choice(2, n_samples)
rainfall = np.random.choice(15, n_samples) 

price_house = 70000 * square + 50000 * number_rooms - 1500 * distance_subway - 3000 * year + 1000 * floor + 60000


data = pd.DataFrame({'square': square, 'number_rooms': number_rooms, 'distance_subway': distance_subway, 'year': year, \
                     'floor' : floor, 'cost_Gazprom_share' : cost_Gazprom_share, 'pet' : pet, 'rainfall' : rainfall, \
                    'price_house' : price_house})
data.head(5)

,square,number_rooms,distance_subway,year,floor,cost_Gazprom_share,pet,rainfall,price_house
0,20,3,1,27,4,384,0,3,1531500
1,38,3,1,6,16,144,0,5,2866500
2,40,3,1,28,24,179,1,9,2948500
3,41,4,2,12,2,197,0,5,3093000
4,60,3,2,37,19,361,1,5,4315000


In [83]:
from sklearn.metrics import mean_absolute_error

##### Построим модель линейной регрессии по всем 8-ми признакам, обучение будем проводить на первых 700 объектах, прогнозирование -  на оставшихся 300 объектах. 
##### Выведем коэффициенты при регрессорах, а также константу, которая показывает смещение по оси ординат.
#####  через метод predict выполним прогнозирование на основе полученной модели.
#####  выведем среднюю абсолютную ошибку полученной модели

In [84]:
X = data[['square', 'number_rooms', 'distance_subway', 'year', 'floor','cost_Gazprom_share', 'pet', 'rainfall']]
y = data['price_house']
reg = LinearRegression().fit(X[:700],y[:700])
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(X[700:])

score = reg.score(X[700:],y[700:])
print('R^2: {}'.format(score))

print('Error: {}'.format(mean_absolute_error(pred_values, y[700:])))

Weights: [ 7.00000000e+04  5.00000000e+04 -1.50000000e+03 -3.00000000e+03
  1.00000000e+03 -3.20142135e-10 -1.13828946e-11  6.02540240e-12]
Bias: 60000.000000108965
R^2: 1.0
Error: 3.887262816230456e-08


In [85]:
y.median()

3094250.0

##### Как мы видим, коэффициенты при ожидаемо лишних регрессорах рассчитались близкими к нулю, поэтому коэффициент детерминации равен 1 и средняя абсолютная ошибка прблизительно равна 0. Однако, несмотря на то что нам повезло на данном примере, мы можем последовательно (или сразу все три) убирать регрессоры, которые не были заложены в расчет зависимой переменной, и оценивать, как это повлияет на качество модели.

In [86]:
# уберем признак rainfall
X = data[['square', 'number_rooms', 'distance_subway', 'year', 'floor','cost_Gazprom_share', 'pet']]
y = data['price_house']
reg = LinearRegression().fit(X[:700],y[:700])
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(X[700:])

print('Error: {}'.format(mean_absolute_error(pred_values, y[700:])))

Weights: [ 7.00000000e+04  5.00000000e+04 -1.50000000e+03 -3.00000000e+03
  1.00000000e+03 -7.27595761e-12 -4.83169060e-12]
Bias: 60000.00000000326
Error: 9.747842947642008e-10


##### Небольшое значение средней абсолютной ошибки еще более уменьшилось, то есть, признак rainfall не улучшал качество модели, значит, он в модели не нужен.

In [89]:
# уберем признаки cost_Gazprom_share и pet
X = data[['square', 'number_rooms', 'distance_subway', 'year', 'floor']]
y = data['price_house']
reg = LinearRegression().fit(X[:700],y[:700])
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(X[700:])

score = reg.score(X[700:],y[700:])
print('R^2: {}'.format(score))

print('Error: {}'.format(mean_absolute_error(pred_values, y[700:])))

Weights: [70000. 50000. -1500. -3000.  1000.]
Bias: 59999.99999999907
R^2: 1.0
Error: 3.927076856295268e-10


##### MAE еще уменьшилась. Как можно видеть, коэффициенты при регрессорах найдены точные, небольшое (статистически незначимое) отклонение есть только в смещении. Можно также провести обучение на всех данных

In [93]:
X = data[['square', 'number_rooms', 'distance_subway', 'year', 'floor']]
y = data['price_house']
reg = LinearRegression().fit(X,y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(X)

score = reg.score(X,y)
print('R^2: {}'.format(score))

print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [70000. 50000. -1500. -3000.  1000.]
Bias: 60000.000000000466
R^2: 1.0
Error: 1.1338852345943451e-10


##### Таким образом, добавление лишних регрессоров ухудшает качество модели или, по крайней мере, не улучшает его. Соответственно, незачем перегружать модель. 